<a href="https://colab.research.google.com/github/GM-DEVELOPER1/teperature-project/blob/main/EDA%2BLSTM_Forecast_Pred%F0%9F%9A%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
subhamjain_temperature_of_all_countries_19952020_path = kagglehub.dataset_download('subhamjain/temperature-of-all-countries-19952020')

print('Data source import complete.')


# Global Warming

### Global warming refers to the rise in global temperatures due mainly to the increasing concentrations of greenhouse gases in the atmosphere

![](https://responsabilidadsocial.net/wp-content/uploads/2018/06/calentamiento-global-que-es-definicion-causas-consecuencias-combate-lucha-contra.jpg)

### The aim of this project is to analyze the global temperature growth and to predict how is going to be in the following years.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# EDA-Exploratory Data Analysis

In [ ]:
#Low memory: The reason you get this low_memory warning is because guessing dtypes for each column is very memory demanding.
temperature_df=pd.read_csv('../input/temperature-of-all-countries-19952020/city_temperature.csv',low_memory=False)
temperature_df.head()

In [ ]:
temperature_df.info()

In [ ]:
temperature_df.isnull().sum() #Looking at Null Vallues

In [ ]:
temperature_df['State'].value_counts() #Looking at this column, we consider that is meaningless for the project porpuse.

In [ ]:
temperature_df=temperature_df.drop('State',axis=1) #Drop column

In [ ]:
world=temperature_df.groupby(['Year','Month','Day'])['AvgTemperature'].mean().reset_index() #Group by, mean temperature for each day and set a DF
world

In [ ]:
world['Year'].value_counts() #Looking at year

In [ ]:
world.drop(world[world['Year']<1995].index,inplace=True) #Drop meaningless rows
world





In [ ]:
world['AvgTemperature'].describe() #Looking at the AvgTemperature

In [ ]:
world[world['AvgTemperature']<0]['AvgTemperature'].describe() #Further look at the negative values, we think there are mistakes

In [ ]:
world.drop(world[world['AvgTemperature']<50].index,inplace=True)

### As we can see the AVG Temperature is in Fahrenheit Scale. We turn it into Celsius Scale just substracting 32 or Adding 32 if the temperature is below freezing.

In [ ]:
world['AvgTemperature']=world['AvgTemperature'].map(lambda x: x-32 if x>0 else x+32) #Celsius
world

In [ ]:
world['Day'].value_counts()

In [ ]:
world.drop(world[world['Day']==0].index,inplace=True) #0 is a mistake

In [ ]:
world['Date']= world['Year'].map(str)+"-"+world['Month'].map(str)+"-"+world['Day'].map(str) #Create a new column with Year,Month,Day

In [ ]:
world['Date'] #Take a look

In [ ]:
import warnings
warnings.filterwarnings('ignore') #avoid Warnings
sns.set_style('darkgrid')
plt.rcParams['figure.figsize']=50,10
plt.title('Average Temperature of the world ',fontsize=15, fontweight='bold')

ax=sns.lineplot(data=world,x='Date',y='AvgTemperature',palette="Blues_d") #lineplot
plt.locator_params(axis='x', nbins=10)#to get the xtick correctly
every_nth = 100
for n, label in enumerate(ax.xaxis.get_ticklabels()):
    if n % every_nth != 0:
        label.set_visible(False)
plt.xticks(rotation=90, fontweight='bold',fontsize=10);

### The line plot is a cyclic data. if you look properly the temperature is going high year by year

In [ ]:
temperature = world.filter(['AvgTemperature'])
# Convert the dataframe to a numpy array
temp_values = temperature.values
# Get the number of rows to train the model on
training_data_len = int(np.ceil( len(temp_values) * .95 )) #get the 95% of rows

training_data_len

In [ ]:
from sklearn.preprocessing import MinMaxScaler #preprocessing

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(temp_values)

scaled_data

In [ ]:
# Create the training data set
# Create the scaled training data set
train_data = scaled_data[0:int(training_data_len), :]
# Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)
        print()

# Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
# x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=50, epochs=1)

In [ ]:
# Create the testing data set
# Create a new array containing scaled values from index 1543 to 2002
test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = temp_values[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

# Convert the data to a numpy array
x_test = np.array(x_test)

# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

# Get the models predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
rmse

In [ ]:
train = temperature[:training_data_len]
valid = temperature[training_data_len:]
valid['Predictions'] = predictions
# Visualize the data
plt.figure(figsize=(16,6))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('AvgTemperature', fontsize=18)
plt.plot(train['AvgTemperature'])
plt.plot(valid[['AvgTemperature', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

## References:


https://analyticsindiamag.com/tutorial-on-univariate-single-step-style-lstm-in-time-series-forecasting/
https://keras.io/examples/timeseries/timeseries_weather_forecasting/

### Hope you like it!!!!